In [1]:
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.linalg as sla
import pickle
import os
from sklearn.datasets import load_svmlight_file

from methods import GradientDescent, Newton, Nesterov, RBFGS, LBFGS, BFGS
from methods.rbfgs import Uniform, Gaussian, CustomDiscrete, ConstantDistribution, \
                          MatrixDistribution
from oracles import LogRegL2Oracle, create_log_reg_oracle, QuadraticOracle
from utils import select_basis_columns, run_rbfgs_experiment, run_all_methods

In [2]:
%%time
file = './datasets/HIGGS'
A, b = load_svmlight_file(file, n_features=None)
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))
A = A.toarray()

(n, d) = (11000000, 28)
density = 0.9210572077922078
CPU times: user 2min 32s, sys: 6.24 s, total: 2min 39s
Wall time: 2min 37s


In [3]:
print('Smoothness based on row norms: {}'.format(np.max(npla.norm(A, axis=1))**2 / 4.))

Smoothness based on row norms: 581.8141043024459


In [13]:
sketch_sizes = [2, 5, 15, 25]
def experiment_higgs(regcoef):
    oracle = create_log_reg_oracle(A, b, regcoef)
    run_all_methods(
        oracle=oracle, 
        sketch_sizes=sketch_sizes, 
        max_iter=300, 
        mat=A, 
        output_folder='./results/higgs/regcoef={:.2e}'.format(regcoef), 
        sigma_tolerance=1e-8, 
        method_tolerance=1e-16, 
        stopping_criteria='grad_abs', 
        random_state=0, 
        methods=['gauss', 'coord', 'svd-no-sigma', 'bfgs', 'nesterov'], 
        overwrite=False
    )

In [14]:
%%time
experiment_higgs(1e-1)

Computing SVD...Done
Singular values above tolerance: 28

RBFGS-SVD sketch no sigma... Done
RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done

All runs completed.
CPU times: user 1h 50min 59s, sys: 6min 30s, total: 1h 57min 30s
Wall time: 4min 26s


### Nesterov

In [5]:
%%time
np.random.seed(42)
regcoef = 1e-1
oracle = create_log_reg_oracle(A, b, regcoef)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/higgs/regcoef={:.2e}'.format(regcoef)
method = Nesterov(oracle, x_0, stopping_criteria='grad_abs', 
                  tolerance=1e-16)
method.run(max_iter=500000, max_time=150)
method.oracle = None
with open('{}/nesterov.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)

CPU times: user 1h 7min 49s, sys: 3min 7s, total: 1h 10min 56s
Wall time: 2min 33s


### Solve logreg

In [6]:
from methods import LBFGS
from tqdm import tqdm_notebook

In [7]:
def solve_logreg(A, b, regcoef, dataset_name, tolerance=1e-16):
    import os
    import pickle
    os.system('mkdir -p ./logreg_solutions/{}'.format(dataset_name))
    
    oracle = create_log_reg_oracle(A, b, regcoef)
    method = LBFGS(oracle, x_0, tolerance=tolerance, stopping_criteria='grad_abs', 
                   memory_size=10, line_search_options={'method': 'Wolfe'})
    method.run(10000)

    with open('./logreg_solutions/{}/regcoef={:.2e}'\
              .format(dataset_name, regcoef), 'wb') as file:
        pickle.dump({'x_star': method.hist['x_star'], 
                     'func_star': method.hist['func'][-1]}, file)

In [8]:
x_0 = np.zeros((A.shape[1]))

In [10]:
%%time
for regcoef in tqdm_notebook([1e-1]):
    solve_logreg(A, b, regcoef, 'higgs', tolerance=1e-18)


CPU times: user 21min, sys: 55.6 s, total: 21min 55s
Wall time: 48.2 s
